Predicting Missing Twitter Handles from Text

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
%matplotlib inline

In [2]:
df=pd.read_csv('winedata.csv')

In [3]:
test=df[df['taster_twitter_handle'].isnull()]   #Test data

In [4]:
df2=df[df['taster_twitter_handle'].notnull()] 

In [5]:
#Splitting into Train and validation sets
from sklearn.model_selection import train_test_split
train,validation=train_test_split(df2,test_size=0.2)

In [6]:
by_handle=train.groupby('taster_twitter_handle')

In [7]:
wordfreq=nltk.probability.ConditionalFreqDist()
for name,group in by_handle:
    tweets=group['description'].str.cat(sep = ' ') 
    #get all of the tweets they wrote and collapse them into a
    #single long string
    tweets=tweets.lower().decode('utf-8')
    #split the text into individual tokens
    tokens=nltk.tokenize.word_tokenize(tweets)
    # calculate the frequency of each token
    freq=nltk.FreqDist(tokens)
    #add the frequencies for each author to our dictionary
    wordfreq[name]=(freq)
    

Guessing who tweeted the review using Joint Probability

In [8]:
def Guess_Joint_Prob(s):
    testTweet=s
    processedTweet=nltk.tokenize.word_tokenize(testTweet.lower())
    #print(processedTweet)
    #testProbabilities=pd.DataFrame(columns=['handle','word','probability'])
    testProbList = []
    for i in wordfreq.keys(): #get handles
        for j in processedTweet: #each word from tweet
            tweetfreq=wordfreq[i].freq(j) #how frequently that word was tweeted by that handle
            smoothedfreq=tweetfreq+0.000001
            #adding handle and smoothedfreq to DF
            d = {
                'handle': i,
                'word': j,
                'probability':smoothedfreq
            }
            testProbList.append(d)
    testProbabilities = pd.DataFrame(data =testProbList, columns=['handle','word','probability'])

    testProbabilitiesbyHandleList = []
    for i in wordfreq.keys():
        oneHandle=testProbabilities.query('handle =="'+ i +'"' )
        #print(oneHandle, i)
        jointProbability=oneHandle.product(numeric_only=True)[0]
        #print(jointProbability)
        d = {
            'handle': i,
            'jointProbability': jointProbability
        }
        testProbabilitiesbyHandleList.append(d)
        #print(testProbabilitiesbyHandle)
    testProbabilitiesbyHandle=pd.DataFrame(data = testProbabilitiesbyHandleList, columns=['handle','jointProbability'])
    return (testProbabilitiesbyHandle.loc[testProbabilitiesbyHandle['jointProbability'].idxmax(),'handle'])

In [9]:
Guess_Joint_Prob('It is a spicy wine with rich notes of plum and apples. The vanilla finish with hints of spices lends a unique taste.')

'@gordone_cellars'

In [10]:
reviews=validation.description.values.tolist()

In [11]:
import sys
reload(sys)  
sys.setdefaultencoding('utf8')

In [12]:
from sklearn import metrics

In [16]:
review_decoded = []
for i in reviews:
    review_decoded.append(str(i).decode('utf-8'))

In [ ]:
%time 
pred_handles=map(Guess_Joint_Prob,review_decoded)

In [ ]:
%%time
metrics.accuracy_score((pred_handles),validation.taster_twitter_handle.values.tolist())